In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [73]:
# for i in range (1, 51):


url = "https://westlafayette.evergreenindiana.org/Search/Results?lookfor=&searchIndex=Keyword&filter[]=format_category%3A%22Books%22&filter[]=availability_toggle%3A%22local%22&sort=relevance&view=list&searchSource=local"
response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')
all_books_on_page = soup.find('div', id="searchInfo")
books = all_books_on_page.find_all('h3')



In [75]:
title = all_books_on_page.find('a')

In [77]:
name = title.attrs['aria-label']

KeyError: 'aria-label'

In [71]:
print(titles)

[]


In [40]:
print(books)

[<h3 style="margin-top:0"><span class="result-index">1)</span> <a aria-label="Three_hundred_years_of_American_painting " class="result-title notranslate" href="/GroupedWork/3bcaec5a-f468-90db-5e29-87890fb8154e-eng/Home?searchId=2533542&amp;recordIndex=1&amp;page=1&amp;searchSource=local&amp;referred=resultIndex">Three hundred years of American painting</a></h3>, <h3 style="margin-top:0"><span class="result-index">2)</span> <a aria-label="Consumer_reports " class="result-title notranslate" href="/GroupedWork/8969895c-8a91-50c3-d95c-d9e4917b9412-eng/Home?searchId=2533542&amp;recordIndex=2&amp;page=1&amp;searchSource=local&amp;referred=resultIndex">Consumer reports</a></h3>, <h3 style="margin-top:0"><span class="result-index">3)</span> <a aria-label="The_Friday_afternoon_club  a_family_memoir" class="result-title notranslate" href="/GroupedWork/13b988e6-ef19-273d-5be0-5ca63934f349-eng/Home?searchId=2533542&amp;recordIndex=3&amp;page=1&amp;searchSource=local&amp;referred=resultIndex">The F

In [33]:
print(all_books_on_page)

<div id="searchInfo">
<div class="topFacets"><br/><div class="formatCategories top-facet" id="formatCategories"><div class="row" id="categoryValues"><div class="categoryValue categoryValue_books col-tn-2"><a class="removeFacetLink" href="/Search/Results?lookfor=&amp;searchIndex=Keyword&amp;filter[]=availability_toggle%3A%22local%22&amp;sort=relevance&amp;view=list&amp;searchSource=local" title="Remove Filter"><div class="row"><div class="col-xs-6"><img alt="Books" src="/interface/themes/responsive/images/books_selected.png"/></div><div class="col-xs-6 formatCategoryLabel">Books<br/>(Remove)</div></div></a></div><div class="categoryValue categoryValue_ebook col-tn-2"><a href="/Search/Results?lookfor=&amp;searchIndex=Keyword&amp;filter[]=format_category%3A%22Books%22&amp;filter[]=format_category%3A%22eBook%22&amp;filter[]=availability_toggle%3A%22local%22&amp;sort=relevance&amp;view=list&amp;searchSource=local"><div class="row"><div class="col-xs-6"><img alt="Filter Format by eBook" src=